# Squared numbers

Create a list (using a loop) and an array (without loop) containing the square of integer number from 0 to N-1. Compare execution speed with $N=10^6$.


In [ ]:
import numpy as np

In [ ]:
%%timeit 
list_square = []
for i in range(1000000):
    list_square.append(i**2)

In [ ]:
%%timeit 
array_square = np.arange(1000000)**2

# Calculation of pi

Without any loop, calculates $\pi$ using the formula :

$$   \pi = \sqrt{12}\sum_{k=0}^{\infty} \frac{(-1)^k}{3^k(2k+1)} $$


In [ ]:
N = 34
Tk = np.arange(N, dtype=np.float64)

print(np.sqrt(12)*sum((-1)**Tk / (3**Tk*(2*Tk+1))))

# Allan variance

The allan variance of a set of measurements $y_k$, where each point corresponds to a frequency measured during $\tau$ is defined as :

$$    \sigma_y^2(\tau) = \frac 12 \left<\left(y_{k+1} - y_k\right)^2\right>_k $$

Write a function that calculates the Allan variance **without any loop**. Check that for white noise, the Allan variance is the same as the variance. Calculate the Allan variance for the following data.


In [ ]:
def allan_variance(y):
    """Allan variance 

    """
    return  ((y[1:] - y[:-1])**2).mean() /2

In [ ]:
N = 10000
t = np.linspace(0, 30, num=N, endpoint=False)

noise = np.random.normal(size=N)
data = 0.1*t + noise

In [ ]:
allan_variance(data)

In [ ]:
data.var()

# Mandelbrot set

The Mandelbrot set is defined as the set of points $c$ of the complex plane such that the following sequence  :

$$    z_0=0$$

$$    z_{n+1}=z_n^2+c $$

is bounded. 

One can show that if there is a value of $n$ such that $|z_n|>2$ the the sequence is divergent. To calculate the Mandelbrot set, for each value of $c$ one calculate $100$ iterations. The picture is plotted by giving to each point the smallest value of $n < 100$ such that $|z_n|>2$ (and $0$ if this value doesn't exist). 

Calculate and plot the Mandelbrot set for $c$ such that $-2.13 < \operatorname{Re}(c) < 0.77$ and $-1.13 < \operatorname{Im}(c) < 1.13$. One can then zoom on the edge of the set.

Note : to plot an image, use the imshow function of pylab. 

The total computation time using numpy efficiently is less than 1 seconds for a 1024x1024 matrix. 

![Mandelbrot set](mandel_couleur.png "mandel_couleur.png")


In [ ]:
from time import time

x_min, x_max = -2.13, 0.77
y_min, y_max = -1.13, 1.13

N = 1024

def mandel_numpy(x_min, x_max, y_min, y_max, N):
    """ Mandelbrot using numpy """
    x = np.linspace(x_min, x_max, N)
    y = np.linspace(y_min, y_max, N)*1j

    c = x + y[:,None]

    image = np.zeros((N,N))

    z = np.zeros_like(c)
    for i in range(100):
        keep = image==0
        z[keep] = z[keep]**2 + c[keep]
        ind = (abs(z)>2) & (image==0)
        image[ind] = i

    return image

def _mandel_suite(c):
    z = 0
    for i in range(100):
        z = z**2 + c
        if abs(z)>2:
            return i
    return 0

mandel_suite = np.vectorize(_mandel_suite)

def mandel_vectorized(x_min, x_max, y_min, y_max, N):
    """ Mandelbrot set using vectorize from numpy """
    x = np.linspace(x_min, x_max, N)
    y = np.linspace(y_min, y_max, N)*1j

    c = x + y[:,None]

    return mandel_suite(c)


import numba

@numba.vectorize
def _mandel_numba(c, N_iter_max=100):
    z = 0
    for n in range(N_iter_max):
        if abs(z)>2:
            return n
        z = z**2 + c
    return 0

def mandel_numba(x_min, x_max, y_min, y_max, N):
    """ Mandelbrot set using vectorize from numpy """
    x = np.linspace(x_min, x_max, N)
    y = np.linspace(y_min, y_max, N)*1j

    c = x + y[:,None]

    return _mandel_numba(c, 100)


t0 = time()
mandel_numpy(x_min, x_max, y_min, y_max, N)
print("Numpy :", time()-t0)
t0 = time()
mandel_vectorized(x_min, x_max, y_min, y_max, N)
print("Vectorize :", time()-t0)
t0 = time()
mandel_numba(x_min, x_max, y_min, y_max, N)
print("Numba :", time()-t0)


In [ ]:
from matplotlib.pyplot import figure

fig = figure()
ax = fig.subplots(1, 1)

ax.imshow(mandel_numba(x_min, x_max, y_min, y_max, N), extent=(x_min, x_max, y_min, y_max))

fig.savefig('mandel_couleur.png')
fig.savefig('mandel_couleur.pdf')

# Measurement of pi (Monte Carlo)

This exercise can be solved without any loop.

* Using the ``rand`` function, creates two array X and Y with ``N=1000`` random samples from a uniform distribution over ``[-1, 1[``.

* Plot the points

* Plot a circle of radius 1 and plot using a different color the points inside the circle.

* How many points are inside the circle ?

* The number of points is proportional to the area of the circle. Deduce an approximate value of $\pi$. One can use $10^6$ points (without plotting the figure...)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 1000
X = 2*np.random.rand(N)-1
Y = 2*np.random.rand(N)-1

plt.plot(X,Y, '.')
cond = (X**2 + Y**2)<1
plt.plot(X[cond],Y[cond], '.')
theta = np.linspace(0, 2*np.pi, 201)
plt.plot(np.sin(theta), np.cos(theta))
ax = plt.gca()
ax.set_aspect('equal')

def mesure_pi(N):
    X = 2*np.random.rand(N)-1
    Y = 2*np.random.rand(N)-1
    nb_points = sum((X**2 + Y**2)<1)
    return 4*nb_points/float(N)
    
print(mesure_pi(1000000))


# Display data from a gravimeter

A gravimeter is a device that measures the acceleration of gravity. In Paris, it is g ∼ 9.81
m/s 2 . In this exercise we will plot data from the cold atom gravimeter
of the LNE-SYRTE laboratory (in Paris) and those coming from a superconductor gravimeter which
were recorded in parallel, during 27 days, between April 7 and May 4, 2015. The cold atom gravimeter
('CAG' for cold atom gravimeter) is a so-called absolute gravimeter in the sense that
it measures g directly in m/s 2 . The superconductor gravimeter ('iGRAV') is a relative gravimeter
relative in the sense that it measures a voltage in V which can be related to g. The latter must therefore
be calibrated. This can be done with the AGC. This is what we are going to do in a first step.
first step.

You will find a text file 'CAGiGRAV.txt' in the data folder of the zip which contains: the date (in
Julian day modified 'MJD'), the measure of the CAG (in $nm/s^2$ , compared to the value g = 9.808 907 500
$m/s^2$ ), the iGRAV measurement (in V) and the residual between the two measurements after calibration of the iGRAV calibration (in nm/s 2 ). This content is separated by 'tabs'. The first line corresponds to the header.

There is a conversion factor for the igrav, which is $K = -898.25 nm/s^2 / V$

Load the data, plot the data for the CAG, the iGRAV as well as there difference (on an independant subplot)

In [ ]:
import numpy as np
from matplotlib.pyplot import figure

MJD, CAG, iGRAV, RES = np.loadtxt('data/CAGiGrav.txt', delimiter='\t', skiprows=1, unpack=True)

fig = figure()
ax1, ax2 = fig.subplots(2, 1, gridspec_kw={'height_ratios':[2,1]}, sharex=True)

ax1.plot(MJD, CAG, label='CAG')
ax1.plot(MJD, -iGRAV*898.25, label='iGRAV')

ax1.legend()
ax1.set_ylabel('nm/s^2')
ax2.plot(MJD, CAG +iGRAV*898.25)
ax2.set_ylabel('nm/s^2')

ax2.set_xlabel('MJD')

# Bode plot

The transfer function of a damped oscillator can written in the Fourier space using the formula : 

$$    H(\omega) = \frac{\omega_0^2}{\omega_0^2 - \omega^2 + 2j\zeta\omega\omega_0} $$


We would like to draw the three following graphs : 

* Bode plot for $\omega_0/2\pi = 1 \mathrm{kHz}$ and $\zeta=0.1$

![Bode plot](bode_1.png "Bode plot")

* Tranfer amplitude function for $\omega_0/2\pi = 1 \mathrm{kHz}$ and different values of $\zeta$.

![Bode plot](bode_2.png "Bode plot")

* Bode plot for the product of two tranfer function $\omega_0/2\pi = 1 \mathrm{kHz}$, $\zeta=0.5$, $\omega_1/2\pi = 2 \mathrm{kHz}$, $\zeta=0.5$.

![Bode plot](bode_3.png "Bode plot")

You will uses the following functions :

* ``loglog``

* ``semilogx``

* ``xlabel``, ``ylabel`` et ``title``

* ``grid`` 

* ``subplot(ny, nx, n)`` 

* The ``label`` optional parameter and the function ``legend``.

* The ``angle`` function of numpy can be used to calculate the phase of a complex number. For the last graph, one should modify the phase in order for the plot to be continuous (do it without any loop!).

You should also know how to 

* format a string to insert parameters

* For the greek letters, one can use unicode or latex formula.


In [ ]:
def H(omega, omega_0, zeta):
    return omega_0**2/(omega_0**2-omega**2 + 2J*omega*zeta*omega_0)


omega_0 = 2*np.pi*1000
zeta = 0.1

Tomega = 2*np.pi*10**(np.linspace(2,4, 1001))
amplitude = H(Tomega, omega_0, zeta)

plt.subplot(2,1,1)
title_str = 'Damped oscillator ($\omega_0/2\pi$={0:.3f} Hz and $\zeta={1:.3f}$)'
plt.title(title_str.format(omega_0/(2*np.pi),zeta))
plt.grid(True, which='both')
plt.loglog(Tomega/(2*np.pi), abs(amplitude))
plt.ylabel('Amplitude')
plt.subplot(2,1,2)
plt.semilogx(Tomega/(2*np.pi), np.angle(amplitude)/(2*np.pi)*360)
plt.ylabel(u'Phase [°]')
plt.xlabel(u'Frequency [Hz]')
plt.grid(True, which='both')

plt.savefig('bode_1.pdf')
plt.savefig('bode_1.png')

In [ ]:
Tzeta = [.03,0.1,0.3,1]
Tlinetype = ['-', '--', '-.', ':']

for zeta, linetype in zip(Tzeta, Tlinetype):
    amplitude = H(Tomega, omega_0, zeta)
    plt.loglog(Tomega/(2*np.pi), abs(amplitude), 'k'+linetype, 
    label="$\zeta={0:4.2f}$".format(zeta), linewidth=2)
plt.grid(True)
plt.xlabel(u'Frequency [Hz]')
plt.ylabel(u'Phase [°]')
plt.legend()
title_str = 'Damped oscillator ($\omega_0/2\pi$={0:.2f} Hz)'
plt.title(title_str.format(omega_0/(2*np.pi)))
plt.savefig('bode_2.pdf')
plt.savefig('bode_2.png')

In [ ]:
omega_0 = 2*np.pi*1000
omega_1 = 2*np.pi*2000

zeta_0 = 0.5
zeta_1 = 0.5

Tomega = 2*np.pi*10**(np.linspace(2,4, 1001))
amplitude = H(Tomega, omega_0, zeta_0)*H(Tomega, omega_1, zeta_1)


plt.subplot(2,1,1)
title_str='Double oscillator ($\omega_0/2\pi$={0:.3f} Hz,\
 $\zeta_0={1:.3f}$, $\omega_1/2\pi$={2:.3f} Hz et $\zeta_1={3:.3f}$)'
plt.title(title_str.format(omega_0/(2*np.pi),zeta_0, omega_1/(2*np.pi), zeta_1))
plt.grid(True, which='both')
plt.loglog(Tomega/(2*np.pi), abs(amplitude))
plt.ylabel('Amplitude')

plt.subplot(2,1,2)
phase = np.unwrap(np.angle(amplitude))
plt.semilogx(Tomega/(2*np.pi), phase/(2*np.pi)*360)
plt.ylabel(u'Phase [°]')
plt.xlabel(u'Frequency [Hz]')
plt.grid(True, which='both')

plt.savefig('bode_3.pdf')
plt.savefig('bode_3.png')

# Fit of inteference fringes

We would like to fit the fringes from an atom interferometer. The data are in the file (``data/fit_sinus.dat``). The first column of the file (x axis) represents a frequency in Hz. The second column (y axis) represents the population measured for the given frequency. The goal is to find the position of the central fringe. 

The fit function is a cosine function with adjustable offset, amplitude, position and width. 

* Load and plot the data. 

* Write the fit function called ``fringe(x, ....)`` 

* Find the initial parameter for the fit. 

* Calculate the optimal parameters

* What is the position and uncertainty of the central fringe ?

In [ ]:
import numpy as np
from matplotlib.pyplot import figure

from scipy.optimize import curve_fit

data = np.loadtxt('data/fit_sinus.dat')

freq = data[:,0]
y = data[:,1]

def fringe(x, offset, amplitude, center, Delta_f):
    "y = offset + amplitude* (1+cos(2*pi*(x-center)/Delta_f))/2"
    return offset + amplitude* (1+np.cos(2*np.pi*(x-center)/Delta_f))/2

param_ini = [0.12, .2, 0., 200.]

fig = figure()
ax = fig.subplots(1, 1)
ax.plot(freq, y, 'o', label='data')

x_fit = np.linspace(min(freq), max(freq))
#plot(x_fit, fringe(x_fit, *param_ini))

popt, cov = curve_fit(fringe, freq, y, p0=param_ini)

offset, amplitude, center, Delta_f = popt
sigma_offset, sigma_amplitude, sigma_center, sigma_Delta_f = np.sqrt(np.diag(cov))

ax.plot(x_fit, fringe(x_fit, *popt), label='fit')

ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Population')
ax.legend()

print(f'Sigma center : {sigma_center}')

# Fit of a picture

We have a 64x64 picture of a double star. Using the example given in the lecture, fit the picture by the sum of two gaussians and get the distance between the two stars.

Data are in the file ``data/double_star.txt``).


In [ ]:
import numpy as np
from matplotlib.pyplot import figure

image = np.loadtxt('data/double_star.txt')

ny, nx = image.shape
X,Y = np.meshgrid(range(nx), range(ny))
# two column matrices with X and Y
XY = np.array([X.flatten(), Y.flatten()]).transpose()

def gauss(XY, amplitude, center_x, center_y, diameter):
    x = XY[:,0]
    y = XY[:,1]
    return amplitude*np.exp(-((x-center_x)**2 + (y-center_y)**2)/diameter**2)

def model(XY, *param):
    return gauss(XY, *param[:4]) + gauss(XY, *param[4:8]) + param[8]

fig = figure()
ax = fig.subplots(1, 1)
ax.imshow(image, interpolation = 'nearest')

# Measured position of the center using the imshow figure.
x0, y0 = 31, 25
x1, y1 = 31, 38

p0 = [200, x0, y0, 2, 200, x1, y1, 2, 0.2]

# One can look at the initial parameters
#image_test = model(XY, *p0).reshape(X.shape)
#ax.imshow(image_test, interpolation = 'nearest')

popt, pcov = curve_fit(model, XY, image.flatten(), p0)
print("Distance : ", np.hypot(popt[1] - popt[5], popt[2] - popt[6]))

fig = figure()
ax = fig.subplots(1, 1)
image_fit = model(XY, *popt).reshape(X.shape)
ax.imshow(image_fit, interpolation = 'nearest')
